<a href="https://colab.research.google.com/github/TrelisResearch/gguf-quantization/blob/main/HuggingFace_to_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Converting HuggingFace Models to GGML

Find Trelis on [HuggingFace](https://huggingface.co/Trelis) and [YouTube](https://www.youtube.com/@TrelisResearch) and [GitHub](https://github.com/TrelisResearch).


### Login to Huggingface Hub

You need to create token with write role [here](https://huggingface.co/settings/tokens) and input in cell below.

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
cache_dir = ''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
cache_dir = "/content/drive/My Drive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists


### Install requirements

In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q einops
!pip install numpy==1.24
!pip install sentencepiece==0.1.98

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.0 which is incompati

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00


In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

### Model loading

In [ ]:
# model_name = 'meta-llama/Llama-2-7b'
model_name = 'Trelis/Llama-2-7b-chat-hf-function-calling-v2'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='cpu',
    offload_folder='offload',
    cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git


Cloning into 'llama.cpp'...
remote: Enumerating objects: 7859, done.
remote: Counting objects: 100% (3759/3759), done.
remote: Compressing objects: 100% (551/551), done.
remote: Total 7859 (delta 3544), reused 3260 (delta 3207), pack-reused 4100
Receiving objects: 100% (7859/7859), 7.38 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (5453/5453), done.


In [ ]:
%cd llama.cpp
# %cd ../


/content/llama.cpp


In [ ]:
model.save_pretrained('./models/')

In [ ]:
!ls -l

total 15612
-rw-r--r--  1 root root     123 Aug 28 08:48 build-info.h
-rw-r--r--  1 root root    5004 Aug 28 08:47 build.zig
drwxr-xr-x  2 root root    4096 Aug 28 08:47 ci
-rw-r--r--  1 root root   23260 Aug 28 08:47 CMakeLists.txt
drwxr-xr-x  2 root root    4096 Aug 28 08:47 common
-rw-r--r--  1 root root   85872 Aug 28 08:48 common.o
-rw-r--r--  1 root root   15856 Aug 28 08:48 console.o
-rwxr-xr-x  1 root root    9345 Aug 28 08:47 convert-falcon-hf-to-gguf.py
-rwxr-xr-x  1 root root    8874 Aug 28 08:47 convert-gptneox-hf-to-gguf.py
-rwxr-xr-x  1 root root  845008 Aug 28 08:49 convert-llama2c-to-ggml
-rwxr-xr-x  1 root root    9910 Aug 28 08:47 convert-llama-7b-pth-to-gguf.py
-rwxr-xr-x  1 root root   15430 Aug 28 08:47 convert-llama-ggmlv3-to-gguf.py
-rwxr-xr-x  1 root root   10547 Aug 28 08:47 convert-llama-hf-to-gguf.py
-rwxr-xr-x  1 root root    4267 Aug 28 08:47 convert-lora-to-ggml.py
-rwxr-xr-x  1 root root   47612 Aug 28 08:47 convert.py
drwxr-xr-x  2 root root    4096 Aug 

# Converting to gguf
!!! Be sure to manually upload tokenizer.model

In [ ]:
!apt update
!apt install build-essential git cmake libopenblas-dev libeigen3-dev

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
16 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
b

In [ ]:
!make LLAMA_OPENBLAS=1

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.            -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS -DGGML_USE_OPENBLAS -I/usr/include/x86_64-linux-gnu/openblas-pthread/
I CXXFLAGS: -I. -I./common -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I LDFLAGS:  -L/usr/lib/x86_64-linux-gnu/openblas-pthread/ -lopenblas
I CC:       cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:      g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

make: Nothing to be done for 'default'.


In [ ]:
!ls

build-info.h			 ggml-alloc.o	   llama.cpp
build.zig			 ggml.c		   llama.h
ci				 ggml-cuda.cu	   llama.o
CMakeLists.txt			 ggml-cuda.h	   main
common				 ggml.h		   Makefile
common.o			 ggml-metal.h	   media
console.o			 ggml-metal.m	   models
convert-falcon-hf-to-gguf.py	 ggml-metal.metal  Package.swift
convert-gptneox-hf-to-gguf.py	 ggml-mpi.c	   perplexity
convert-llama2c-to-ggml		 ggml-mpi.h	   pocs
convert-llama-7b-pth-to-gguf.py  ggml.o		   prompts
convert-llama-ggmlv3-to-gguf.py  ggml-opencl.cpp   quantize
convert-llama-hf-to-gguf.py	 ggml-opencl.h	   quantize-stats
convert-lora-to-ggml.py		 gguf		   README.md
convert.py			 gguf-py	   requirements.txt
docs				 grammar-parser.o  scripts
embd-input-test			 grammars	   server
embedding			 k_quants.c	   SHA256SUMS
examples			 k_quants.h	   simple
flake.lock			 k_quants.o	   spm-headers
flake.nix			 libembdinput.so   tests
ggml-alloc.c			 LICENSE	   train-text-from-scratch
ggml-alloc.h			 llama-bench	   vdot


In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
!python convert.py models/

In [ ]:
!./quantize ./ggml-model-f16.bin ./ggml-model-q3_K_M.bin q3_K_M

In [ ]:
!./quantize models/ggml-model-f16.gguf models/ggml-model.gguf Q4_K_M

main: build = 1100 (dd0dc36)
main: quantizing 'models/ggml-model-f16.gguf' to 'models/ggml-model.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 16 key-value pairs and 291 tensors from models/ggml-model-f16.gguf (version GGUF V1 (support until nov 2023))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight f16      [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight f16      [  4096, 1

### Push to Hub
Need to write this out